In [1]:
import pandas as pd
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import numpy as np
import numutils
from mpl_toolkits import mplot3d
from scipy import signal
%matplotlib widget

In [2]:
df = pd.read_excel('aaf8084_SupportingFile_Suppl1._Excel_seq4_v1.xlsx')

In [3]:
df.columns = ['serial', 'ID', 'x', 'y', 'z']

In [4]:
df.head()

,serial,ID,x,y,z
0,1,1,20.058508,27.677535,2.680473
1,1,2,20.910140,26.994707,3.056548
2,1,3,20.551936,27.291391,2.051425
3,1,4,20.097117,27.537659,2.757642
4,1,5,20.337958,27.218279,2.600930


In [5]:
M= list()
for s in df.serial.unique():
    Z = df[df.serial == s]
    if np.sum(np.sum(Z.isna() == True)) > 0:
        continue
    r = Z.iloc[:,2:].values
    r = signal.resample(r , 1000)
    arr = numutils.distance(r, norm = 'L2')
    W, B, stats =  numutils.correction(arr)
    M.append(W/W.sum(axis = 0, keepdims = True))

In [6]:
M =  np.array(M)

In [7]:
S = M[0].sum(axis = 0)

In [8]:
arr = M.mean(axis = 0)

In [9]:
plt.figure()
plt.imshow(arr, cmap = 'YlOrRd')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
O = numutils.expected(arr)

In [11]:
plt.figure()
plt.imshow(O, cmap = 'YlOrRd')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
enrich = numutils.enrichment(arr) - 1
corr = np.corrcoef(enrich)

In [13]:
plt.figure(figsize= (10, 3))
plt.subplot(121)
plt.title('Enrichment')
plt.hist(enrich.reshape(-1))
plt.subplot(122)
plt.title('Pearson Correlation')
plt.hist(corr.reshape(-1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 58260., 145140., 130306., 114980., 111214., 105828., 106714.,
        100380.,  68906.,  58272.]),
 array([-0.84627374, -0.66164636, -0.47701899, -0.29239162, -0.10776424,
         0.07686313,  0.26149051,  0.44611788,  0.63074525,  0.81537263,
         1.        ]),
 <a list of 10 Patch objects>)

In [14]:
plt.figure(figsize = (10, 4))
plt.subplot(121)
plt.title('Enrichment')
plt.imshow(enrich, cmap = 'bwr', interpolation = 'none', vmin = -0.5, vmax = 0.5)
plt.subplot(122)
plt.title('Pearson Correlation')
plt.imshow(corr, cmap = 'bwr', vmin = -1, vmax = 1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
S, E = np.linalg.eig(corr)

In [16]:
np.round(S/np.sum(np.abs(S)), 2)[:10]

array([0.44, 0.12, 0.11, 0.09, 0.04, 0.03, 0.03, 0.02, 0.02, 0.02])

In [17]:
plt.figure(figsize = (10, 3))
V = -E[:,0]
pos = V > 0
x = np.linspace(0, len(V)-1, len(V), dtype = int)
plt.bar(x[~pos],V[~pos]) #plot negatives
plt.bar(x[pos],V[pos])   #plot positives
plt.margins(0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.figure(figsize = (10, 3))
plt.subplot(121)
plt.title('Pearson Correlation')
plt.imshow(corr, cmap = 'bwr', interpolation = 'bilinear', vmin = -1, vmax = 1)
plt.subplot(122)
plt.title('E1 outerproduct')
plt.imshow(S[0]*np.outer(V, V), cmap = 'bwr', interpolation = 'bilinear', vmin = -0.2, vmax = 0.2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
test = df[df.serial == 4].iloc[:, 2:].values

In [20]:
'''
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(test.x, test.y, test.z, c=pos, cmap='bwr');
ax.plot3D(test.x, test.y, test.z, 'gray');
plt.show()
'''

'\nfig = plt.figure()\nax = plt.axes(projection="3d")\nax.scatter3D(test.x, test.y, test.z, c=pos, cmap=\'bwr\');\nax.plot3D(test.x, test.y, test.z, \'gray\');\nplt.show()\n'

In [21]:

r = test
r = signal.resample(test , 1000)
fig = plt.figure()
ax = plt.axes(projection="3d")
ax.scatter3D(r[:,0], r[:,1], r[:, 2],c=V, cmap='bwr');
ax.plot3D(r[:,0], r[:,1], r[:, 2], 'gray');
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …